In [ ]:
!pip install pandas
import pandas as pd



Index(['County', 'GEOID', 'anglo', 'anglovap', 'asian', 'asianvap', 'bh',
       'bhvap', 'black', 'blackvap', 'density', 'hisp', 'hispvap', 'nanglo',
       'nanglovap', 'official_boundary', 'pct_dem_lead', 'vap', 'votes_dem',
       'votes_rep', 'votes_total', 'state', 'total', '2026_dem_raw_vote',
       '2026_rep_raw_vote', 'total_26', 'area_sqmi'],
      dtype='object')
25106371.0
20842.0


In [9]:
df = pd.read_csv("TX_24_with_counties_with_area.csv")
print(df.columns)
print(len(df))
print(df["vap"].max())
df_edu = pd.read_csv("TX_24_with_edu_vtd_pop.csv")
# print(df_edu.columns)
print(df_edu[['CNTY', 'COLOR', 'VTD', 'CNTYKEY', 'VTDKEY', 'CNTYVTD','CNTYVTD_y']].head(10))
print(df_edu["vap"].sum())
print(len(df_edu))
print(len(df))

Index(['County', 'GEOID', 'anglo', 'anglovap', 'asian', 'asianvap', 'bh',
       'bhvap', 'black', 'blackvap', 'density', 'hisp', 'hispvap', 'nanglo',
       'nanglovap', 'official_boundary', 'pct_dem_lead', 'vap', 'votes_dem',
       'votes_rep', 'votes_total', 'state', 'total', '2026_dem_raw_vote',
       '2026_rep_raw_vote', 'total_26', 'area_sqmi'],
      dtype='object')
9718
20842.0
   CNTY  COLOR   VTD  CNTYKEY  VTDKEY CNTYVTD CNTYVTD_y
0     1      4  0001        1     1.0   10001   0010001
1     1      3  0002        1     2.0   10002   0010002
2     1      6  0003        1     3.0   10003   0010003
3     1      2  0004        1     4.0   10004   0010004
4     1      1  0005        1     5.0   10005   0010005
5     1      2  0006        1     6.0   10006   0010006
6     1      6  0007        1     7.0   10007   0010007
7     1      1  0008        1     8.0   10008   0010008
8     1      4  0009        1     9.0   10009   0010009
9     1      1  0010        1    10.0   10010   0

In [24]:
#!/usr/bin/env python3
"""
Add CNTYVTD_y from messy GEOID values in a CSV.
- County code has no leading zeros
- VTD padded to 4 digits with optional trailing letter
- Dallas rules:
    * 48113-####-02 → dropped
    * 48113-####-01 → keep #### (strip -01)
"""

import re
import pandas as pd

# ==== CHANGE THESE PATHS ====
in_path = "TX_24_with_counties_with_area.csv"
out_path = "TX_24_with_counties_with_area_with_cntyvtd.csv"
# =============================


def parse_cntyvtd(geoid_raw: str):
    """
    Convert various GEOID formats into CNTYVTD_y.
    Returns (cntyvtd, action), where action is 'keep' or 'drop'.
    """
    s = (geoid_raw or "").strip()
    if not s:
        return None, "keep"

    # Dallas special cases
    if s.startswith("48113-"):
        m_da = re.fullmatch(r"48113-([0-9A-Za-z]+)-(0[12])", s)
        if m_da:
            if m_da.group(2) == "02":
                return None, "drop"
            s = f"48113-{m_da.group(1)}"  # strip -01

    # County: prefer leading 5 digits (take last 3), else first 3 consecutive digits
    county = None
    m5 = re.match(r"^\s*(\d{5})", s)
    if m5:
        county = str(int(m5.group(1)[2:]))  # strip leading zeros
    else:
        m3 = re.search(r"(\d{3})", s)
        if m3:
            county = str(int(m3.group(1)))  # strip leading zeros

    # Precinct side
    right = s.split("-", 1)[1] if "-" in s else s
    m_vtd = re.search(r"(?i)(?:precinct\s*)?(\d+)\s*([A-Za-z]?)", right)
    if not m_vtd:
        m_vtd = re.search(r"(\d{1,4})([A-Za-z]?)", right)

    if not county or not m_vtd:
        return None, "keep"

    vtd_num = m_vtd.group(1)
    vtd_suffix = (m_vtd.group(2) or "").upper()

    try:
        vtd_int = int(vtd_num)
    except ValueError:
        return None, "keep"

    vtd_padded = f"{vtd_int:04d}"
    cntyvtd = f"{county}{vtd_padded}{vtd_suffix}"
    return cntyvtd, "keep"


def main():
    df = pd.read_csv(in_path, dtype=str, keep_default_na=False)

    cntyvtd_vals = []
    keep_mask = []
    for val in df.get("GEOID", [""]*len(df)):
        cntyvtd, action = parse_cntyvtd(val)
        cntyvtd_vals.append(cntyvtd)
        keep_mask.append(action != "drop")

    df["CNTYVTD_new"] = cntyvtd_vals
    out_df = df[keep_mask].copy()  # drop Dallas -02 rows

    # (Optional) drop rows with no CNTYVTD_y:
    # out_df = out_df[out_df["CNTYVTD_y"].notna()].copy()

    out_df.to_csv(out_path, index=False)

    total = len(df)
    kept = len(out_df)
    dropped = total - kept
    derived = int(out_df["CNTYVTD_new"].notna().sum())
    print({
        "rows_in": total,
        "rows_out": kept,
        "rows_dropped_dallas_-02": dropped,
        "rows_with_cntyvtd_y": derived
    })
    print(out_df["CNTYVTD_new"].head(10))


if __name__ == "__main__":
    main()


{'rows_in': 9718, 'rows_out': 9468, 'rows_dropped_dallas_-02': 250, 'rows_with_cntyvtd_y': 9468}
0    10001
1    10002
2    10003
3    10004
4    10005
5    10006
6    10007
7    10008
8    10009
9    10010
Name: CNTYVTD_new, dtype: object


In [54]:
with_edu = pd.read_csv("TX_24_with_edu_vtd_pop.csv")
with_edu["CNTYVTD_new"] = (
    with_edu["CNTY"].astype(str).str.strip() +
    with_edu["VTD"].astype(str).str.strip()
)
place = pd.read_csv("TX_24_with_counties_with_area_with_cntyvtd.csv")
with_area_new = place[["CNTYVTD_new", "GEOID"]]
print(len(with_area_new))

merged_edu_with_area = with_edu.merge(with_area_new, on="CNTYVTD_new", how="left")
unmatched = merged_edu_with_area.isna()
unmatched.to_csv("unmatched.csv", index=False)
print(f"Total rows in with_edu: {len(with_edu)}")
print(f"Unmatched rows after merge: {unmatched}")
print(f"Matched rows: {len(with_edu) - unmatched}")

print("vote_dem total merged_edu_with_area", merged_edu_with_area["votes_dem"].sum())



9468
Total rows in with_edu: 9712
Unmatched rows after merge:       Unnamed: 0   CNTY  COLOR    VTD  CNTYKEY  VTDKEY  CNTYVTD  CNTYVTD_y  \
0          False  False  False  False    False   False    False      False   
1          False  False  False  False    False   False    False      False   
2          False  False  False  False    False   False    False      False   
3          False  False  False  False    False   False    False      False   
4          False  False  False  False    False   False    False      False   
...          ...    ...    ...    ...      ...     ...      ...        ...   
9873       False  False  False  False    False   False    False      False   
9874       False  False  False  False    False   False    False      False   
9875       False  False  False  False    False   False    False      False   
9876       False  False  False  False    False   False    False      False   
9877       False  False  False  False    False   False    False      False   

 

In [ ]:

# print(merged_edu_with_area.columns)
# print(df.columns)
df_margin = df[["pct_dem_lead", "GEOID"]]
test = merged_edu_with_area.merge(df_margin, on="GEOID", how="left")
test["college_enroll"] = test["ASPSE018_alloc_sum"] + test["ASPSE017_alloc_sum"]
# print(test.columns)
test.to_csv("test_1.csv", index=False)

print(merged_edu_with_area["vap"].sum())
print(with_edu["vap"].sum())

print(merged_edu_with_area["ASPSE018_alloc_sum"].sum())
print(with_edu["ASPSE018_alloc_sum"].sum())
print(merged_edu_with_area["ASPSE017_alloc_sum"].sum())
print(with_edu["ASPSE017_alloc_sum"].sum())




In [58]:
edu_geo = pd.read_csv("TX_24_geoid_edu.csv")
print(edu_geo.columns)
edu_geo["college_enroll"] = edu_geo["ASPSE018_sum"] + edu_geo["ASPSE017_sum"]
print(edu_geo["ASPSE018_sum"].sum())
print(edu_geo["ASPSE017_sum"].sum())
edu_geo["college_enroll"] = edu_geo["college_enroll"].round(0)
edu_geo["bachelors_up_total_sum"] = edu_geo["bachelors_up_total_sum"].round(0)

pre_merge = edu_geo[["GEOID", "college_enroll", "bachelors_up_total_sum"]]
df = pd.read_csv("TX_24_with_counties_with_area.csv")

df_edu = df.merge(pre_merge, on="GEOID", how="left")
print(df_edu.columns)
df_edu.to_csv("test_2.csv", index=False)


Index(['Unnamed: 0', 'state', 'GEOID', 'votes_dem', 'votes_rep', 'votes_total',
       'pct_dem_lead', 'official_boundary', 'County', 'total', 'anglo',
       'nanglo', 'asian', 'black', 'hisp', 'bh', 'vap', 'anglovap',
       'nanglovap', 'asianvap', 'blackvap', 'hispvap', 'bhvap', 'density',
       'prec_id', 'ASPSE017_sum', 'ASPSE018_sum', 'bachelors_up_total_sum'],
      dtype='object')
374111.2045632639
1447461.22359058
Index(['County', 'GEOID', 'anglo', 'anglovap', 'asian', 'asianvap', 'bh',
       'bhvap', 'black', 'blackvap', 'density', 'hisp', 'hispvap', 'nanglo',
       'nanglovap', 'official_boundary', 'pct_dem_lead', 'vap', 'votes_dem',
       'votes_rep', 'votes_total', 'state', 'total', '2026_dem_raw_vote',
       '2026_rep_raw_vote', 'total_26', 'area_sqmi', 'college_enroll',
       'bachelors_up_total_sum'],
      dtype='object')
